In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "qwen2.5"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, dtype=torch.float16)
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


1

In [2]:
from datasets import load_dataset
train_dataset = load_dataset("json", data_files="train_data.json", split='train')
print(train_dataset.shape)

(945, 1)


In [3]:
from transformers import TrainingArguments

per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
logging_steps = 10
learning_rate = 1e-4
max_grad_norm = 0.3
num_train_epochs = 3
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    num_train_epochs=num_train_epochs,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
)


In [4]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
)

In [ ]:
trainer.train()

Step,Training Loss
10,5.292900
20,4.066300
30,3.743200
40,3.527000
50,3.523700
60,3.607100
70,2.958200
80,2.932700
90,2.786300
100,2.902300


In [ ]:
model.save_pretrained("./qwen-ccb")
tokenizer.save_pretrained("./qwen-ccb")